<a href="https://colab.research.google.com/github/Zarina33/Titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FILE_TRAIN = r'/content/drive/MyDrive/Kaggle/titanic/train.csv'

FILE_VALIDATION = r'/content/drive/MyDrive/Kaggle/titanic/test.csv'
data = pd.read_csv(FILE_TRAIN)
test = pd.read_csv(FILE_VALIDATION)
test_ids = test["PassengerId"]

In [ ]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#for cleaning from trash

data.drop(["PassengerId", "Name", "Ticket","Cabin","Embarked","SibSp","Parch"], axis = 1, inplace = True)


In [ ]:
data.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [ ]:
data.isnull().sum()



Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
dtype: int64

In [ ]:
data['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [ ]:
data['Age'].fillna(data['Age'].mean(), inplace = True)

In [ ]:
data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
dtype: int64

In [ ]:
l_sex_dummies = pd.get_dummies(data['Sex'], drop_first=True)

In [ ]:
data = pd.concat([data, l_sex_dummies], axis = 1)


In [ ]:
data.head()

,Survived,Pclass,Sex,Age,Fare,male
0,0,3,male,22.0,7.2500,1
1,1,1,female,38.0,71.2833,0
2,1,3,female,26.0,7.9250,0
3,1,1,female,35.0,53.1000,0
4,0,3,male,35.0,8.0500,1


In [ ]:
data.drop(['Sex'], axis = 1, inplace = True)

In [ ]:
data.head()

,Survived,Pclass,Age,Fare,male
0,0,3,22.0,7.2500,1
1,1,1,38.0,71.2833,0
2,1,3,26.0,7.9250,0
3,1,1,35.0,53.1000,0
4,0,3,35.0,8.0500,1


In [ ]:
from sklearn.preprocessing import StandardScaler
sts = StandardScaler()

In [ ]:
feature_scale = ['Age','Fare']
data[feature_scale] = sts.fit_transform(data[feature_scale])

In [ ]:
data.head()

,Survived,Pclass,Age,Fare,male
0,0,3,-0.592481,-0.502445,1
1,1,1,0.638789,0.786845,0
2,1,3,-0.284663,-0.488854,0
3,1,1,0.407926,0.420730,0
4,0,3,0.407926,-0.486337,1


In [ ]:
X = data.drop(['Survived'], axis= 1)
y = data['Survived']

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
#create param

model_param = {
    'DecisionTreeClassifier':{
        'model':DecisionTreeClassifier(),
        'param':{
            'criterion':['gini', 'entropy']
        }
    },

    'KNeighborsClassifier':{
        'model':KNeighborsClassifier(),
        'param':{
            'n_neighbors':[5,10,15,20,25]
        }
    },
    'SVC':{
        'model':SVC(),
        'param':{
            'kernel':['rbf','linear','sigmoid'],
            'C':[0.1,1,10,100]
        }
    }
}

In [ ]:
scores = []
for model_name, mp in model_param.items():
  model_selection = GridSearchCV(estimator = mp['model'], param_grid = mp['param'],cv= 5, return_train_score = False )
  model_selection.fit(X,y)
  scores.append({
      'model':model_name,
      'best_score':model_selection.best_score_,
      'best_params': model_selection.best_params_
  })

In [ ]:
data_model_score = pd.DataFrame(scores,columns=['model', 'best_score', 'best_params'])
data_model_score

,model,best_score,best_params
0,DecisionTreeClassifier,0.777861,{'criterion': 'gini'}
1,KNeighborsClassifier,0.803616,{'n_neighbors': 5}
2,SVC,0.811481,"{'C': 100, 'kernel': 'rbf'}"


In [ ]:
model_svc = SVC( C = 100, kernel = 'rbf')

In [ ]:
model_svc.fit(X,y)

SVC(C=100)

In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
test2=test.drop(['PassengerId','Name','Ticket','Embarked','SibSp','Cabin','Parch'], axis = 1)

In [ ]:
test2.isnull().sum()

Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

In [ ]:
test2['Age'].fillna(test2['Age'].mean(), inplace = True)
test2['Fare'].fillna(test2['Fare'].mean(), inplace = True)

In [ ]:
l_sex_dummies=pd.get_dummies(test2['Sex'], drop_first=True)
test2 = pd.concat([test2,l_sex_dummies], axis = 1)
test2.drop(['Sex'], axis = 1, inplace = True)

In [ ]:
test2.head()

,Pclass,Age,Fare,male
0,3,34.5,7.8292,1
1,3,47.0,7.0000,0
2,2,62.0,9.6875,1
3,3,27.0,8.6625,1
4,3,22.0,12.2875,0


In [ ]:
test2[feature_scale] = sts.fit_transform(test2[feature_scale])

In [ ]:
test2.head()

,Pclass,Age,Fare,male
0,3,0.334993,-0.498407,1
1,3,1.325530,-0.513274,0
2,2,2.514175,-0.465088,1
3,3,-0.259330,-0.483466,1
4,3,-0.655545,-0.418471,0


In [ ]:
y_predicted = model_svc.predict(test2)

In [ ]:
submission = pd.DataFrame({
    "PassengeId":test['PassengerId'],
    "Survived":y_predicted
})

In [ ]:
submission.to_csv('titanic_submission', index = False)